# Fake News Classifier : Deep Learning

Libraries: 

    Link: https://www.kaggle.com/c/fake-news/data
    Dataset: 
        id: unique id for a news article
        title: the title of a news article
        author: author of the news article
        text: the text of the article; could be incomplete
        label: a label that marks the article as potentially unreliable
            1: unreliable
            0: reliable

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
print(df.shape)

(20800, 5)


In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
# Drop Nan Values
df = df.dropna()

In [7]:
X = df.drop('label',axis=1)
Y = df['label']

In [8]:
print(X.shape, Y.shape)

(18285, 4) (18285,)


In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.3.0'

In [11]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [12]:
# Vocabulary Size
voc_size = 5000

In [13]:
messages = X.copy()

In [14]:
messages['title'][1:3]

1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
Name: title, dtype: object

In [15]:
# we have Dropped Nan values before so, reset_index() method sets a list of 
# integer ranging from 0 to length of data as index
messages.reset_index(inplace=True)

### Data Preprocessing 

In [16]:
import nltk 
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\aakriti
[nltk_data]     aggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    # print(i)
    review = re.sub('[^a-zA-z]',' ', messages['title'][i])
    # substituting everything with blank except alphabets
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
# First for each index sentence, except all the alphabets all the words will be subsituted to ' '
# then conver each word into lowercase and split each word separately in the sentence.
# for each word not present in stopwords then only that word is stored in review
# again join the words that were splitted for forming a sentence and append together each sentence

In [19]:
corpus[:5]
# all the non necessary word is removed

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

### One Hot Representation

In [20]:
onehotRep = [one_hot(words, voc_size) for words in corpus]
print(onehotRep[:5])

[[2177, 1069, 2453, 420, 2446, 3839, 2411, 2959, 4623, 2965], [3346, 2427, 2126, 353, 1265, 2904, 2693], [2446, 19, 711, 3808], [4559, 3158, 4612, 967, 4600, 1215], [37, 1265, 4246, 180, 4514, 4549, 1265, 741, 4745, 4459]]


### Embedding Representation

In [21]:
# All the sent have variable len; using pad_sequence fix the len
sent_len = 20
embedded_docs = pad_sequences(onehotRep, padding = 'pre', maxlen = sent_len)
print(embedded_docs[:5])

[[   0    0    0    0    0    0    0    0    0    0 2177 1069 2453  420
  2446 3839 2411 2959 4623 2965]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0 3346
  2427 2126  353 1265 2904 2693]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 2446   19  711 3808]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
  4559 3158 4612  967 4600 1215]
 [   0    0    0    0    0    0    0    0    0    0   37 1265 4246  180
  4514 4549 1265  741 4745 4459]]


In [22]:
len(embedded_docs)

18285

## Creating Model

In [23]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_len))
model.add(LSTM(100)) # 1 LSTM layer having 100 neurons
model.add(Dense(1,activation='sigmoid')) # coz of classification prob
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [25]:
print(X_final.shape, Y_final.shape)

(18285, 20) (18285,)


In [26]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_final, Y_final, 
                                                test_size = 0.3, random_state=42)

## Model Training

In [27]:
model.fit(Xtrain,Ytrain,validation_data = (Xtest,Ytest), epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 9s 46ms/step - loss: 0.3324 - accuracy: 0.8441 - val_loss: 0.1918 - val_accuracy: 0.9182
Epoch 2/10
200/200 [==============================] - 8s 42ms/step - loss: 0.1328 - accuracy: 0.9501 - val_loss: 0.1971 - val_accuracy: 0.9192
Epoch 3/10
200/200 [==============================] - 8s 42ms/step - loss: 0.0994 - accuracy: 0.9631 - val_loss: 0.2122 - val_accuracy: 0.9167
Epoch 4/10
200/200 [==============================] - 8s 42ms/step - loss: 0.0724 - accuracy: 0.9746 - val_loss: 0.2354 - val_accuracy: 0.9110
Epoch 5/10
200/200 [==============================] - 8s 42ms/step - loss: 0.0525 - accuracy: 0.9824 - val_loss: 0.3155 - val_accuracy: 0.9154
Epoch 6/10
200/200 [==============================] - 8s 41ms/step - loss: 0.0369 - accuracy: 0.9881 - val_loss: 0.3641 - val_accuracy: 0.9131
Epoch 7/10
200/200 [==============================] - 8s 41ms/step - loss: 0.0274 - accuracy: 0.9915 - val_loss: 0.3632 - val_accuracy: 0.9141

In [28]:
# Since the accuracy is 0.9996 almost 1.0 it may lead to overfitting prob

## Performance Metrics and Accuracy

In [29]:
y_pred = model.predict_classes(Xtest)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Ytest,y_pred)
# False + and False - are very less compare to True + and True -

array([[2884,  223],
       [ 293, 2086]], dtype=int64)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(Ytest,y_pred)

0.9059423988333941

## Adding Dropout (Optional)

In [32]:
from tensorflow.keras.layers import Dropout
## creating Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_len))
model.add(LSTM(150)) # 1 LSTM layer having 100 neurons
model.add(Dropout(0.6))
model.add(Dense(1,activation='sigmoid')) # coz of classification prob
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               114600    
_________________________________________________________________
dropout (Dropout)            (None, 150)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 151       
Total params: 314,751
Trainable params: 314,751
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
model.fit(Xtrain,Ytrain,validation_data = (Xtest,Ytest), epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 12s 60ms/step - loss: 0.3300 - accuracy: 0.8458 - val_loss: 0.1988 - val_accuracy: 0.9158
Epoch 2/10
200/200 [==============================] - 10s 51ms/step - loss: 0.1414 - accuracy: 0.9491 - val_loss: 0.2015 - val_accuracy: 0.9178
Epoch 3/10
200/200 [==============================] - 10s 52ms/step - loss: 0.0973 - accuracy: 0.9644 - val_loss: 0.2148 - val_accuracy: 0.9174
Epoch 4/10
200/200 [==============================] - 10s 51ms/step - loss: 0.0767 - accuracy: 0.9733 - val_loss: 0.2526 - val_accuracy: 0.9147
Epoch 5/10
200/200 [==============================] - 10s 52ms/step - loss: 0.0505 - accuracy: 0.9830 - val_loss: 0.2844 - val_accuracy: 0.9118
Epoch 6/10
200/200 [==============================] - 10s 52ms/step - loss: 0.0349 - accuracy: 0.9891 - val_loss: 0.3924 - val_accuracy: 0.9043
Epoch 7/10
200/200 [==============================] - 11s 53ms/step - loss: 0.0262 - accuracy: 0.9922 - val_loss: 0.4232 - val_accuracy: